# Installation :


In [1]:
!git clone https://github.com/scikit-mobility/scikit-mobility.git
!cd scikit-mobility && python3 setup.py install
!pip3 install scikit-mobility
!git clone https://github.com/IhabBendidi/morocco.geojson.git

Cloning into 'scikit-mobility'...
remote: Enumerating objects: 258, done.
remote: Counting objects: 100% (258/258), done.
remote: Compressing objects: 100% (174/174), done.
remote: Total 1983 (delta 145), reused 168 (delta 84), pack-reused 1725
Receiving objects: 100% (1983/1983), 30.78 MiB | 21.12 MiB/s, done.
Resolving deltas: 100% (1217/1217), done.
running install
running bdist_egg
running egg_info
creating scikit_mobility.egg-info
writing scikit_mobility.egg-info/PKG-INFO
writing dependency_links to scikit_mobility.egg-info/dependency_links.txt
writing requirements to scikit_mobility.egg-info/requires.txt
writing top-level names to scikit_mobility.egg-info/top_level.txt
writing manifest file 'scikit_mobility.egg-info/SOURCES.txt'
writing manifest file 'scikit_mobility.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/skmob
copying skmob/__init__.py -> build/lib/skm

Cloning into 'morocco.geojson'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 21 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (21/21), done.


Be sure to restart the runtime afterward (on google colab) so that the installation really take place.

In [0]:
import geopandas as gpd  
import pandas as pd
import skmob
from skmob.models.epr import DensityEPR
from skmob.models.epr import Ditras
from skmob.models.markov_diary_generator import MarkovDiaryGenerator
from skmob.preprocessing import filtering, compression, detection, clustering

Setting up the start time of the simulation, between 1/1/2019 and 15/1/2019

In [0]:
# Helper function for protecting against duplicates ids
def change_uid(x,l):
  x = str(x) + l
  return x

In [0]:
# starting and end times of the simulation
start_time = pd.to_datetime('2019/01/01 08:00:00')
end_time = pd.to_datetime('2019/01/15 08:00:00')

### Irfane simulation

In [4]:
# Getting irfane data (rabat)
irfane_tessellation = gpd.read_file('morocco.geojson/irfane.geojson')
depr = DensityEPR()
#generating and filtering data to make it as close to real as possible
density_tdf = depr.generate(start_time, end_time, irfane_tessellation, relevance_column='population', n_agents=120)
density_tdf = filtering.filter(density_tdf, max_speed_kmh=5.)
density_tdf = detection.stops(density_tdf, stop_radius_factor=0.5, minutes_for_a_stop=50.0, spatial_radius_km=0.3, leaving_time=False)

/usr/local/lib/python3.6/dist-packages/scikit_mobility-1.0-py3.6.egg/skmob/models/gravity.py:43: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.6/dist-packages/scipy/sparse/lil.py:504: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/usr/local/lib/python3.6/dist-packages/scipy/sparse/lil.py:506: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):
/usr/local/lib/python3.6/dist-packages/scipy/sparse/lil.py:510: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not x.flags.writeable:


In [0]:
# changing ids of citizens to not have duplicated later on
density_tdf["uid"] = density_tdf["uid"].apply(change_uid,args=["a"])

We will now generate habits of agents, through the markov diary generator, for Al Irfane

In [0]:
# load and preprocess data to train the MarkovDiaryGenerator
url = 'https://raw.githubusercontent.com/scikit-mobility/scikit-mobility/master/tutorial/data/geolife_sample.txt.gz'
df = pd.read_csv(url, sep=',', compression='gzip')
tdf = skmob.TrajDataFrame(df, latitude='lat', longitude='lon', user_id='user', datetime='datetime')
#ctdf = compression.compress(tdf)
stdf = detection.stops(tdf,stop_radius_factor=0.5, minutes_for_a_stop=50.0, spatial_radius_km=0.3, leaving_time=False)
markov_tdf = clustering.cluster(stdf)

In [7]:
# instantiate and train the MarkovDiaryGenerator
mdg = MarkovDiaryGenerator()
mdg.fit(markov_tdf, 2, lid='cluster')

100%|██████████| 2/2 [00:00<00:00,  3.39it/s]


In [8]:
# instantiate the model
ditras = Ditras(mdg)
# run the model
ditras_tdf = ditras.generate(start_time, end_time, irfane_tessellation, relevance_column='population',
                    n_agents=150, od_matrix=None, show_progress=True)

  0%|          | 0/150 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/scikit_mobility-1.0-py3.6.egg/skmob/models/gravity.py:43: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.6/dist-packages/scipy/sparse/lil.py:504: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/usr/local/lib/python3.6/dist-packages/scipy/sparse/lil.py:506: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):
/usr/local/lib/python3.6/dist-packages/scipy/sparse/lil.py:510: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not x.flags.writeable:
100%|██████████| 150/150 [00:

In [0]:
# changing ids of citizens to not have duplicated later on
ditras_tdf["uid"] = ditras_tdf["uid"].apply(change_uid,args=["b"])

In [0]:
irfan_tdf = pd.concat([ditras_tdf,density_tdf],ignore_index=True)

In [0]:
irfan_tdf.to_csv("irfan_mobility.csv")

### Agdal Simulation

In [12]:
# Getting Agdal data (rabat)
agdal_tessellation = gpd.read_file('morocco.geojson/agdal.geojson')
depr = DensityEPR()
#generating and filtering data to make it as close to real as possible
density_tdf = depr.generate(start_time, end_time, agdal_tessellation, relevance_column='population', n_agents=120)
density_tdf = filtering.filter(density_tdf, max_speed_kmh=5.)
density_tdf = detection.stops(density_tdf, stop_radius_factor=0.5, minutes_for_a_stop=50.0, spatial_radius_km=0.3, leaving_time=False)

/usr/local/lib/python3.6/dist-packages/scikit_mobility-1.0-py3.6.egg/skmob/models/gravity.py:43: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.6/dist-packages/scipy/sparse/lil.py:504: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/usr/local/lib/python3.6/dist-packages/scipy/sparse/lil.py:506: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):
/usr/local/lib/python3.6/dist-packages/scipy/sparse/lil.py:510: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not x.flags.writeable:


In [0]:
# changing ids of citizens to not have duplicated later on
density_tdf["uid"] = density_tdf["uid"].apply(change_uid,args=["c"])

In [14]:
# load and preprocess data to train the MarkovDiaryGenerator
url = 'https://raw.githubusercontent.com/scikit-mobility/scikit-mobility/master/tutorial/data/geolife_sample.txt.gz'
df = pd.read_csv(url, sep=',', compression='gzip')
tdf = skmob.TrajDataFrame(df, latitude='lat', longitude='lon', user_id='user', datetime='datetime')
#ctdf = compression.compress(tdf)
stdf = detection.stops(tdf,stop_radius_factor=0.5, minutes_for_a_stop=50.0, spatial_radius_km=0.3, leaving_time=False)
markov_tdf = clustering.cluster(stdf)
# instantiate and train the MarkovDiaryGenerator
mdg = MarkovDiaryGenerator()
mdg.fit(markov_tdf, 2, lid='cluster')

100%|██████████| 2/2 [00:00<00:00,  3.45it/s]


In [15]:
# instantiate the model
ditras = Ditras(mdg)
# run the model
ditras_tdf = ditras.generate(start_time, end_time, agdal_tessellation, relevance_column='population',
                    n_agents=150, od_matrix=None, show_progress=True)

  0%|          | 0/150 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/scikit_mobility-1.0-py3.6.egg/skmob/models/gravity.py:43: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.6/dist-packages/scipy/sparse/lil.py:504: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/usr/local/lib/python3.6/dist-packages/scipy/sparse/lil.py:506: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):
/usr/local/lib/python3.6/dist-packages/scipy/sparse/lil.py:510: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not x.flags.writeable:
100%|██████████| 150/150 [00:

In [0]:
# changing ids of citizens to not have duplicated later on
ditras_tdf["uid"] = ditras_tdf["uid"].apply(change_uid,args=["d"])

In [0]:
agdal_tdf = pd.concat([ditras_tdf,density_tdf],ignore_index=True)

In [0]:
agdal_tdf.to_csv("agdal_mobility.csv")

### Aggregate results

In [19]:
# Getting Agdal data (rabat)
dual_tessellation = gpd.read_file('morocco.geojson/agdal_irfane.geojson')
depr = DensityEPR()
#generating and filtering data to make it as close to real as possible
density_tdf = depr.generate(start_time, end_time, dual_tessellation, relevance_column='population', n_agents=120)
density_tdf = filtering.filter(density_tdf, max_speed_kmh=5.)
density_tdf = detection.stops(density_tdf, stop_radius_factor=0.5, minutes_for_a_stop=50.0, spatial_radius_km=0.3, leaving_time=False)

/usr/local/lib/python3.6/dist-packages/scikit_mobility-1.0-py3.6.egg/skmob/models/gravity.py:43: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.6/dist-packages/scipy/sparse/lil.py:504: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/usr/local/lib/python3.6/dist-packages/scipy/sparse/lil.py:506: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):
/usr/local/lib/python3.6/dist-packages/scipy/sparse/lil.py:510: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not x.flags.writeable:


In [0]:
# changing ids of citizens to not have duplicated later on
density_tdf["uid"] = density_tdf["uid"].apply(change_uid,args=["e"])

In [21]:
# load and preprocess data to train the MarkovDiaryGenerator
url = 'https://raw.githubusercontent.com/scikit-mobility/scikit-mobility/master/tutorial/data/geolife_sample.txt.gz'
df = pd.read_csv(url, sep=',', compression='gzip')
tdf = skmob.TrajDataFrame(df, latitude='lat', longitude='lon', user_id='user', datetime='datetime')
#ctdf = compression.compress(tdf)
stdf = detection.stops(tdf,stop_radius_factor=0.5, minutes_for_a_stop=50.0, spatial_radius_km=0.3, leaving_time=False)
markov_tdf = clustering.cluster(stdf)
# instantiate and train the MarkovDiaryGenerator
mdg = MarkovDiaryGenerator()
mdg.fit(markov_tdf, 2, lid='cluster')

100%|██████████| 2/2 [00:00<00:00,  3.45it/s]


In [22]:
# instantiate the model
ditras = Ditras(mdg)
# run the model
ditras_tdf = ditras.generate(start_time, end_time, dual_tessellation, relevance_column='population',
                    n_agents=150, od_matrix=None, show_progress=True)

  0%|          | 0/150 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/scikit_mobility-1.0-py3.6.egg/skmob/models/gravity.py:43: RuntimeWarning: divide by zero encountered in power
/usr/local/lib/python3.6/dist-packages/scipy/sparse/lil.py:504: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/usr/local/lib/python3.6/dist-packages/scipy/sparse/lil.py:506: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):
/usr/local/lib/python3.6/dist-packages/scipy/sparse/lil.py:510: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not x.flags.writeable:
100%|██████████| 150/150 [00:

In [0]:
# changing ids of citizens to not have duplicated later on
ditras_tdf["uid"] = ditras_tdf["uid"].apply(change_uid,args=["f"])

In [0]:
dual_tdf = pd.concat([ditras_tdf,density_tdf],ignore_index=True)

In [0]:
simulation_tdf = pd.concat([agdal_tdf,irfan_tdf,dual_tdf],ignore_index=True)

In [0]:
simulation_tdf.to_csv('simulation_data.csv')

# Data cleaning and preparation

In [0]:
import pandas as pd
simulation_tdf = pd.read_csv('simulation_data.csv')

In [0]:
import json
from shapely.geometry import shape, Point

In [0]:
# load GeoJSON file containing sectors
with open('morocco.geojson/agdal_irfane.geojson', 'r') as f:
  js = json.load(f)

In [0]:
def get_zone_id(x):
  point = Point( x["lng"],x["lat"])
  zone_id = ""
  for feature in js['features']:
    polygon = shape(feature['geometry'])
    if polygon.contains(point):
      zone_id = feature['properties']['id']
  x['zone_id'] = zone_id
  return x

In [0]:
# Adding the ids of the zones 
simulation_tdf = simulation_tdf.apply(get_zone_id,axis=1)

# Filtering zones without a registered zone id
simulation_tdf = simulation_tdf[simulation_tdf['zone_id'] != ""][["uid","datetime","lat","lng","zone_id"]]

In [0]:
from datetime import datetime
def get_date_precision(x):
  x['datetime_clipped'] = datetime.strptime(x['datetime'].split(':')[0]+":"+x['datetime'].split(':')[1], '%Y-%m-%d %H:%M')#:%M:%S')
  return x

In [0]:
# Clip datetime by the hour
simulation_tdf = simulation_tdf.apply(get_date_precision,axis=1)

In [0]:
import geopy.distance

def get_distance(x):
  coords_1 = (x['lat_x'], x['lng_x'])
  coords_2 = (x['lat_y'], x['lng_y'])
  km_distance = geopy.distance.distance(coords_1, coords_2).km
  x['distance'] = km_distance * 1000
  return x

In [0]:
merged_tdf = simulation_tdf.merge(simulation_tdf, how='left',on=["datetime_clipped","zone_id"])

# Filter to not keep couples if they have same id
merged_tdf = merged_tdf[merged_tdf['uid_x'] != merged_tdf['uid_y']]

# Compute distance between each two citizens
merged_tdf = merged_tdf.apply(get_distance,axis=1)

# Filter for distances lower than 7 meters between citizens
merged_tdf = merged_tdf[merged_tdf['distance']< 7]

In [0]:
# Getting users around in each time 
temp_tdf = merged_tdf.groupby(['uid_x','datetime_clipped','zone_id'])['uid_y'].apply(','.join).reset_index()
temp_tdf = temp_tdf.rename(columns = {'uid_y':'ble_users'})
ble_tdf = merged_tdf[['uid_x','datetime_x','lat_x','lng_x','zone_id','datetime_clipped']].merge(temp_tdf,how='inner', on=['uid_x','datetime_clipped','zone_id']).drop_duplicates()
ble_tdf = ble_tdf.rename(columns = {'uid_x':'uid','datetime_x':'datetime','lat_x':'lat','lng_x':'lng'})

# Delete the first second of the simulation, and last second of the simulation, as a lot of people are near each other in it
ble_tdf = ble_tdf[ble_tdf['datetime'] != '2019-01-01 08:00:00.000000']
ble_tdf = ble_tdf[ble_tdf['datetime'] != '2019-01-15 08:00:00.000000']

In [0]:
ble_tdf.to_csv('full_mobility_data.csv')